In [ ]:
from gs_quant.instrument import IRSwaption, IRSwap
from gs_quant.risk import IRDeltaParallel, IRDelta
from gs_quant.common import AggregationLevel
from gs_quant.session import Environment, GsSession

In [ ]:
# external users should substitute their client id and secret; please skip this step if using internal jupyterhub
GsSession.use(client_id=None, client_secret=None, scopes=('run_analytics',))

### Solve for Delta
Size swap to match swaption delta

In [ ]:
# create 1y5y payer swaption and calculate delta
payer_swaption = IRSwaption('Pay', '5y', 'EUR', expiration_date='1y', buy_sell='Sell')
swaption_delta = payer_swaption.calc((IRDeltaParallel, IRDeltaParallelLocalCcy))

In [ ]:
# solve for $ delta as IRDeltaParallel is in USD
hedge1 = IRSwap('Receive', '5y', 'USD', fixed_rate='atm', notional_amount='${}/bp'.format(swaption_delta[IRDeltaParallel]))
swap_delta = hedge1.calc(IRDeltaParallel)

# check that delta is approximately equal
swaption_delta[IRDeltaParallel] - swap_delta

In [ ]:
# solve for local ccy delta
hedge2 = IRSwap('Receive', '5y', 'EUR', fixed_rate='atm', notional_amount='{}/bp'.format(swaption_delta[IRDeltaParallelLocalCcy]))
parallel_local_delta = IRDelta(aggregation_level=AggregationLevel.Type, currency='local')
swap_delta_local = hedge2.calc(parallel_local_delta)

#check that delta is approximately equal
swaption_delta[parallel_local_delta] - swap_delta_local